# Transform data
Pre-process data records used in original research articles
- assign column types, clean
- rename, reorder columns
- merge tables on keys

In [1]:
import pandas as pd
import numpy as np
import re

Text cleaning helper functions

In [2]:
def remove_nonascii(text):
    return "".join(i for i in text if ord(i)<128)

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

**PAPERS** from ICPSR Bibliography (DBInfo)

In [ ]:
df_paper = pd.read_excel('../data_original/ICPSR_bib_studies_20211111.xlsx', sheet_name=0)

df_paper = df_paper[df_paper['STUD_NUMS'].notna()] # remove papers that do not have study numbers

df_paper['REF_ID'] = df_paper['REF_ID'].astype('int')
df_paper['TITLE'] = df_paper['TITLE'].str.strip() # remove whitespace from original record entry
df_paper['AUTHORS'] = df_paper['AUTHORS'].str.strip()
df_paper['SEC_TITLE'] = df_paper['SEC_TITLE'].str.strip()
df_paper['RIS_TYPE'] = df_paper['RIS_TYPE'].str.strip()
df_paper['RIS_TYPE'] = df_paper['RIS_TYPE'].astype('category')
df_paper['FUNDER'] = df_paper['FUNDER'].str.strip()
df_paper['YEAR_PUB'] = df_paper['YEAR_PUB'].fillna(0).astype('int')
df_paper['DATE_INPUT'] = df_paper['DATE_INPUT'].astype('datetime64[ns]')
df_paper['STUD_NUMS'] = df_paper['STUD_NUMS'].astype('str')

df_paper.loc[df_paper['YEAR_PUB']<1963,'YEAR_PUB'] = 0  # replace publication year for any item published before ICPSR was established

df_paper = df_paper.rename(columns={'STUD_NUMS':'STUDY_NUMS'})

df_paper = df_paper[['REF_ID',
                     'DOI',
                     'TITLE',
                     'AUTHORS',
                     'SEC_TITLE',
                     'RIS_TYPE',
                     'FUNDER',
                     'YEAR_PUB',
                     'DATE_INPUT',
                     'SERIES_NUMS',
                     'STUDY_NUMS']]

df_paper.info()

In [ ]:
df_dimensions = pd.read_csv('../data_original/enriched_bibliography.csv') # Dimensions run on 2021/11/09

df_dimensions = df_dimensions.rename(columns={'doi':'DOI',
                                              'id':'DIM_ID',
                                              'authors':'DIM_AUTH_AFFIL',
                                              'abstract':'DIM_ABSTRACT',
                                              'linkout':'DIM_LINKOUT'})

df_dimensions.info()

In [ ]:
df_paper_merge = df_paper.merge(df_dimensions, on='DOI', how='left') # merge on paper DOIs
df_paper_merge.to_csv('../data_transform/ICPSR_PAPERS.csv',index=False)
df_paper_merge.info()

In [ ]:
df_paper_merge.sample(5)

**STUDIES** from ICPSR catalog (DBInfo)

In [ ]:
df_study = pd.read_excel('../data_original/ICPSR_bib_studies_20211111.xlsx', sheet_name=1)
df_study = df_study[df_study['PERMIT']=='AVAILABLE'] # remove studies that are not publicly available
df_study = df_study[df_study['OBJECTTYPE']=='study'] # remove union catalog entries

df_study['DESCRIPTION'] = df_study['DESCRIPTION_1'].astype('str') \
    + " " + df_study['DESCRIPTION_2'].astype('str') \
    + " " + df_study['DESCRIPTION_3'].astype('str') + " " \
    + df_study['DESCRIPTION_4'].astype('str') \
    + " " + df_study['DESCRIPTION_5'].astype('str') # combine study description to a single field

df_study = df_study.drop(columns=['PERMIT',
                                  'OBJECTTYPE',
                                  'ALTTITLE1',
                                  'ALTTITLE2',
                                  'ALTTITLE3',
                                  'ALTTITLE4',
                                  'ALTTITLE5',
                                  'ALTTITLE6',
                                  'ALTTITLE7',
                                  'DESCRIPTION_1',
                                  'DESCRIPTION_2',
                                  'DESCRIPTION_3',
                                  'DESCRIPTION_4',
                                  'DESCRIPTION_5'])

df_study['STUDY'] = df_study['STUDY'].astype('int')
df_study['NAME'] = df_study['NAME'].str.strip()
df_study['SERIES'] = df_study['SERIES'].astype('str').astype('float')
df_study['SERIES_TITLE'] = df_study['SERIES_TITLE'].str.strip()
df_study['OWNER'] = df_study['OWNER'].astype('category')
df_study['FUNDINGAGENCY'] = df_study['FUNDINGAGENCY'].str.strip()
df_study['DOI'] = df_study['DOI'].str.strip()
df_study['GEO'] = df_study['GEO'].str.strip()
df_study['TERMS'] = df_study['TERMS'].str.strip()
df_study['ORIGRELDATE'] = df_study['ORIGRELDATE'].fillna(0).astype('datetime64[ns]')
df_study['MEMSERV_PI'] = df_study['MEMSERV_PI'].str.strip()
df_study['DESCRIPTION'] = df_study['DESCRIPTION'].str.strip().astype('str')

df_study['DESCRIPTION'] = df_study['DESCRIPTION'].apply(func=remove_nonascii) # remove nonascii
df_study['DESCRIPTION'] = df_study['DESCRIPTION'].apply(func=remove_html) # remove html tags
df_study['DESCRIPTION'] = df_study['DESCRIPTION'].replace(regex=["nan"], value="") # remove string "nan"

df_study = df_study.rename(columns={'FUNDINGAGENCY':'FUNDING_AGENCY','ORIGRELDATE':'RELEASE_DATE','MEMSERV_PI':'PRINCIPAL_INV'})
df_study.info()

Study table: usage

In [ ]:
df_usage = pd.read_excel('../data_original/ICPSR_combined_study_usage_20210430_onlyData_noAllREST.xlsx',sheet_name=0)
df_usage = df_usage.drop(columns=['DATA_PULL_DATE','ORIGRELDATE','OWNER','OWNER_ICPSR','MEMBER','DAYSINSAMPLE_TO20210430','DAYSINSAMPLE_TO20151231','RECENCY','SERIESYN','VARS','SAMPLING','PROPORTIONREST','TOT_DATA', 'NUMTERMS'])

df_usage['STUDY'] = df_usage['STUDY'].astype('int')
df_usage['SINGLEPI'] = df_usage['SINGLEPI'].astype('int')
df_usage['INST_PI'] = df_usage['INST_PI'].astype('int')
df_usage['TOT_PI'] = df_usage['TOT_PI'].astype('int')
df_usage['SDA'] = df_usage['SDA'].astype('int')
df_usage['QTEXT'] = df_usage['QTEXT'].astype('float')
df_usage['SSVD'] = df_usage['SSVD'].astype('float')
df_usage['USERS_2017_TO_PULLDATE'] = df_usage['USERS_2017_TO_PULLDATE'].astype('float')
df_usage['DATAUSERS_2017_TO_PULLDATE'] = df_usage['DATAUSERS_2017_TO_PULLDATE'].astype('float')
df_usage['HAS_RESTRICTED'] = df_usage['HAS_RESTRICTED'].astype('int')
df_usage['ALL_RESTRICTED'] = df_usage['ALL_RESTRICTED'].astype('int')

df_usage = df_usage.rename(columns={'USERS_2017_TO_PULLDATE':'USERS_TO_20210511','DATAUSERS_2017_TO_PULLDATE':'DATAUSERS_TO_20210511','SINGLEPI':'SINGLE_PI'})
df_usage.info()

Study table: curation level

In [ ]:
df_curation = pd.read_csv('../data_original/jira_curation_level_20210426_by_study.csv')
df_curation = df_curation.drop(columns=['cur_lev_rank'])
df_curation['study'] = df_curation['study'].astype('int')
df_curation['curation_level'] = df_curation['curation_level'].astype('category')
df_curation = df_curation.rename(columns={'study':'STUDY','curation_level':'CURATION_LEVEL'})
df_curation.info()

Study table: citations (count number of study citations from Bibliography)

In [ ]:
df_paper['STUDY'] = df_paper['STUDY_NUMS'].str.split(";") # split STUD_NUMS so that each row is one study
df_paper = df_paper.explode('STUDY')
df_paper['STUDY'] = df_paper['STUDY'].astype('int')

df_citations = pd.DataFrame(df_paper['STUDY'].value_counts()).reset_index()
df_citations = df_citations.rename(columns={'count':'CITATIONS_TO_20211116'})
df_citations.info()

Study table: variable counts

In [ ]:
df_variables = pd.read_csv('../data_original/variable_counts_by_study.csv')
df_variables['STUDY'] = df_variables['STUDY'].astype('int')
df_variables['TOTALVARS'] = df_variables['TOTALVARS'].astype('int')
df_variables = df_variables.rename(columns={'TOTALVARS':'TOTAL_VARS'})
df_variables.info()

Merge STUDY tables

In [ ]:
df_study_merge = df_study.merge(df_usage, on='STUDY', how='left') \
    .merge(df_curation, on='STUDY', how='left') \
    .merge(df_citations, on='STUDY', how='left') \
    .merge(df_variables, on='STUDY', how='left') # merge all tables on main list of studies

df_study_merge = df_study_merge[['STUDY', 
                               'DOI',
                               'NAME',
                               'SERIES',
                               'SERIES_TITLE',
                               'PRINCIPAL_INV',
                               'DESCRIPTION',
                               'RELEASE_DATE',
                               'FUNDING_AGENCY',
                               'GEO',
                               'TERMS',
                               'OWNER',
                               'CURATION_LEVEL',
                               'SINGLE_PI',
                               'INST_PI',
                               'TOT_PI',
                               'TOTAL_VARS',
                               'SDA',
                               'QTEXT',
                               'SSVD',
                               'HAS_RESTRICTED',
                               'ALL_RESTRICTED',
                               'USERS_TO_20210511',
                               'DATAUSERS_TO_20210511',
                               'CITATIONS_TO_20211116']]

df_study_merge.to_csv('../data_transform/ICPSR_STUDIES.csv',index=False)
df_study_merge.info()

In [ ]:
df_study_merge.sample(5)

**PREDICTED CURATION ACTIVITIES** from Jira work logs

In [ ]:
df_action = pd.read_csv('../data_original/predicted_curation_by_study.csv')
df_action = df_action.drop(columns=['Unnamed: 0','Desc']) # remove sentence index and work descriptions (identifiable)
df_action = df_action.rename(columns={'Studies':'STUDY','Action':'ACTION_LABEL','Log_hrs':'LOG_HRS','Study_hrs':'STUDY_HRS'})
df_action['STUDY'] = df_action['STUDY'].str.replace("s","") # remove "s" from study number
df_action['STUDY'] = df_action['STUDY'].str.strip()
df_action['STUDY'] = df_action['STUDY'].astype('int')
df_action['ACTION_LABEL'] = df_action['ACTION_LABEL'].str.strip()
df_action['ACTION_LABEL'] = df_action['ACTION_LABEL'].astype('category')
df_action.to_csv('../data_transform/ICPSR_CURATION_LOGS.csv',index=False)
df_action.info()

In [ ]:
df_action.head(5)

**PROCESSING HISTORY COMMANDS** from SPSS processing history files

Will not include in deposit

In [ ]:
# df_processing = pd.read_csv('../data_original/processing_history_commands_2019_20220502.csv')
# df_processing = df_processing.drop(columns=['filename','savedate'])
# df_processing.columns = [x.upper() for x in df_processing.columns]
# df_processing['STUDY'] = df_processing['STUDY'].astype('int')
# df_processing['TOTAL_LINES'] = df_processing['TOTAL_LINES'].astype('int')
# df_processing['COMMENTS'] = df_processing['COMMENTS'].astype('int')
# df_processing.to_csv('../data_transform/ICPSR_PROCESSING_HISTORY.csv',index=False)

# for column in df_processing.columns:
#     print(column)

In [ ]:
# df_processing.sample(5)